

```python
# --- Logistic Regression Run ---
with mlflow.start_run(run_name="LogisticRegression_Baseline"):
    # Create the full pipeline:
    # 1. Preprocess the data
    # 2. Apply SMOTE to the training data *only*
    # 3. Train the model
    lr_pipeline = ImbPipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('classifier', LogisticRegression(random_state=42))
    ])
    
    # Train the model
    lr_pipeline.fit(X_train, y_train)
    
   

### **Code Cell 6: Train Model 2 (Challenger: XGBoost)**

Now we train our more complex model, `XGBClassifier`. The setup is identical, which shows the power of pipelines.

```python
# --- XGBoost Run ---
with mlflow.start_run(run_name="XGBoost_Challenger"):
    
    # Create the full pipeline
    xgb_pipeline = ImbPipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('classifier', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'))
    ])
    
    # Train the model
    xgb_pipeline.fit(X_train, y_train)
    
    # Make predictions
    y_pred = xgb_pipeline.predict(X_test)
    
    # Calculate metrics
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    # Log parameters to MLflow
    mlflow.log_param("model_type", "XGBClassifier")
    mlflow.log_param("smote_enabled", "True")
    
    # Log metrics to MLflow
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("precision_score", precision)
    mlflow.log_metric("recall_score", recall)
    
    # Log the model itself
    mlflow.sklearn.log_model(xgb_pipeline, "model")
    
    print("\n--- XGBoost Results ---")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
```

-----

### **Final Step: Check Your MLflow UI**

After you run these cells, go back to your **MLflow UI browser tab** (at `http://127.0.0.1:5000`) and **hit refresh**.

You should now see your "Proactive Churn Prediction" experiment. Click it, and you'll find two runs:

  * `LogisticRegression_Baseline`
  * `XGBoost_Challenger`

You can click each one to see the parameters (like `model_type`) and metrics (like `f1_score`) that we logged. This is the "experiment tracking" part of the project, and it's a very impressive skill to show.

We have successfully trained, compared, and logged our models\! Phase 3 is complete.

## **Phase 3: Model Training & Tracking**.

This is the "machine learning" part. We'll use the clean `model_input.csv` to train and compare two models: a simple baseline (Logistic Regression) and a more powerful one (XGBoost).

Most importantly, we'll use **MLflow** to track every experiment, which is a critical MLOps skill.

-----

### **Preparation: Install New Packages**

Before starting the notebook, we need to install the packages for this phase. Go to your **terminal** (with the `(venv)` active) and run this:

```bash
pip install mlflow xgboost imbalanced-learn scikit-learn
```

Once installed, **update your `requirements.txt` file**. Open it and add these new lines so it looks like this:

```
pandas
numpy
scikit-learn
jupyterlab
mlflow
xgboost
imbalanced-learn
```

-----

#### **Step 1: Launch MLflow UI**

In your terminal, run this command. This starts the MLflow tracking server.

```bash
mlflow ui
```

This will open a new browser tab (usually at `http://127.0.0.1:5000`). Keep this tab open. It's currently empty, but our experiments will appear here.

-----

#### **Step 2:  Imports and Setup**


In [5]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

# Load the clean data
df_model_input = pd.read_csv("data/model_input.csv")

# Set the MLflow tracking URI (this points to the server we just started)
# This links our script to the MLflow UI

mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Set a name for our experiment
mlflow.set_experiment("Proactive Churn Prediction")

print("Libraries imported and MLflow experiment set.")
df_model_input.head()

Libraries imported and MLflow experiment set.


,user_id,tenure_days,days_since_last_event,total_support_tickets,has_billing_issue,plan_type,billing_cycle,churn
0,1,419,8,0.0,0.0,Basic,Annual,False
1,2,299,2,0.0,0.0,Basic,Monthly,False
2,3,341,37,1.0,0.0,Basic,Monthly,False
3,4,330,51,0.0,0.0,Basic,Annual,False
4,5,443,19,1.0,0.0,Basic,Annual,False


#### **Step 3:  Define Features (X) and Target (y)**

In [6]:
# 'user_id' is an identifier, not a feature
# 'churn' is our target
df_model_input = df_model_input.drop('user_id', axis=1)

# Define our features (X) and target (y)
X = df_model_input.drop('churn', axis=1)
y = df_model_input['churn']

# Identify which columns are categorical and which are numerical
categorical_features = ['plan_type', 'billing_cycle']
numerical_features = ['tenure_days', 'days_since_last_event', 'total_support_tickets', 'has_billing_issue']

print("Features and target defined.")
X.head()

Features and target defined.


,tenure_days,days_since_last_event,total_support_tickets,has_billing_issue,plan_type,billing_cycle
0,419,8,0.0,0.0,Basic,Annual
1,299,2,0.0,0.0,Basic,Monthly
2,341,37,1.0,0.0,Basic,Monthly
3,330,51,0.0,0.0,Basic,Annual
4,443,19,1.0,0.0,Basic,Annual


#### **Step 4:  Create the Preprocessing Pipeline**

This is a crucial step. We create a `ColumnTransformer` to automatically apply scaling to numerical features and one-hot encoding to categorical features.

In [7]:
# Create the transformer for numerical features

numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

# Create the transformer for categorical features

categorical_transformer = Pipeline(
    steps=[("onehot", OneHotEncoder(handle_unknown= "ignore"))]
)

# Combine these transformers into a single preprocessor

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_features),
        ("cat", categorical_transformer,categorical_features)
    ])

print("Preprocessing pipeline created")

Preprocessing pipeline created


#### **Step 5: Split the data**

We split our data into training and testing sets. Notice `stratify=y`—this is **essential** for imbalanced datasets to ensure both sets get the same percentage of churners.


In [8]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")
print(f"Training set churn rate: {y_train.mean():.4f}")
print(f"Testing set churn rate: {y_test.mean():.4f}")

Training set shape: (8000, 6)
Testing set shape: (2000, 6)
Training set churn rate: 0.0481
Testing set churn rate: 0.0480


#### **Step 6: Train Model 1 (Baseline: Logistic Regression)**

Here is our first experiment. We use `mlflow.start_run()` to log everything. We also add `SMOTE()` to our pipeline to handle the class imbalance.

In [10]:
# --- Logistic Regression Run ---
with mlflow.start_run(run_name="LogisticRegression_Baseline"):
    # Create the full pipeline:
    # 1. Preprocess the data
    # 2. Apply SMOTE to the training data *only*
    # 3. Train the model
    lr_pipeline = ImbPipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('classifier', LogisticRegression(random_state=42))
    ])
    
    # Train the model
    lr_pipeline.fit(X_train, y_train)
    
    # Make predictions
    y_pred = lr_pipeline.predict(X_test)
    
    # Calculate metrics
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    # Log parameters to MLflow
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_param("smote_enabled", "True")
    
    # Log metrics to MLflow
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("precision_score", precision)
    mlflow.log_metric("recall_score", recall)
    
    # Log the model itself
    mlflow.sklearn.log_model(lr_pipeline, "model")
    
    print("--- Logistic Regression Results ---")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

2025/10/18 21:36:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/18 21:37:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


--- Logistic Regression Results ---
F1 Score: 0.5073
Precision: 0.3522
Recall: 0.9062
🏃 View run LogisticRegression_Baseline at: http://127.0.0.1:5000/#/experiments/178087657089640150/runs/a071ecf6b87b42b5806c9bac24048d50
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/178087657089640150


### ** Step 7: Train Model 2 (Challenger: XGBoost)**

Now we train our more complex model, `XGBClassifier`. The setup is identical, which shows the power of pipelines.


In [11]:
# --- XGBoost Run ---
with mlflow.start_run(run_name="XGBoost_Challenger"):
    
    # Create the full pipeline
    xgb_pipeline = ImbPipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('classifier', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'))
    ])
    
    # Train the model
    xgb_pipeline.fit(X_train, y_train)
    
    # Make predictions
    y_pred = xgb_pipeline.predict(X_test)
    
    # Calculate metrics
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    # Log parameters to MLflow
    mlflow.log_param("model_type", "XGBClassifier")
    mlflow.log_param("smote_enabled", "True")
    
    # Log metrics to MLflow
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("precision_score", precision)
    mlflow.log_metric("recall_score", recall)
    
    # Log the model itself
    mlflow.sklearn.log_model(xgb_pipeline, "model")
    
    print("\n--- XGBoost Results ---")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

C:\Users\Chethan Vakiti\proactive_churn_project\venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [21:42:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2025/10/18 21:42:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/18 21:42:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



--- XGBoost Results ---
F1 Score: 0.7707
Precision: 0.7248
Recall: 0.8229
🏃 View run XGBoost_Challenger at: http://127.0.0.1:5000/#/experiments/178087657089640150/runs/671601d8c7674c188193c47eb56462e4
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/178087657089640150




| Metric | Logistic Regression (Baseline) | XGBoost (Challenger) | Change |
| :--- | :--- | :--- | :--- |
| **F1 Score** | 0.5073 | **0.7707** | **+51.9%** 📈 |
| **Precision** | 0.3522 | **0.7248** | **+105.8%** 🚀 |
| **Recall** | 0.9062 | **0.8229** | -9.1% 📉 |

### Analysis

This is a huge win. Here's the business translation:

* Our **Recall** is still strong (82%). We are still catching the vast majority of churners.
* Our **Precision** *more than doubled*. This means we've massively reduced the number of "false positives." We are no longer wasting a huge portion of our retention budget on happy customers.
* The **F1 Score** (the balance between them) shows that **XGBoost is the clear winner**. It provides a much more efficient and practical business solution.

---



### Phase 3 Summary

In this phase, **we** successfully trained and evaluated our machine learning models, moving from raw data to actionable intelligence.

1.  **Environment Upgraded:** We installed the necessary ML libraries (`xgboost`, `mlflow`, `imbalanced-learn`) and set up the **MLflow UI**, a professional tool for tracking our experiments.
2.  **Preprocessing Pipeline:** We built a `scikit-learn` pipeline to automatically **scale** our numerical data and **one-hot encode** our categorical data, ensuring our model receives clean, standardized input.
3.  **Handling Imbalance:** We integrated **SMOTE** (Synthetic Minority Over-sampling Technique) into our pipeline. This fixed our class imbalance problem by creating synthetic "churner" examples for the model to learn from.
4.  **Model Experimentation:** We trained and logged two different models:
    * **Baseline (Logistic Regression):** This gave us a starting point, showing high recall but very low precision.
    * **Challenger (XGBoost):** This model was a clear winner, dramatically improving our precision while maintaining high recall, resulting in a much higher F1-score.
5.  **Tracking & Comparison:** We used **MLflow** to log every parameter and metric for both runs. This allows us to compare our models side-by-side and definitively prove *why* XGBoost is the superior model for this business problem.

We now have a high-performing, tracked, and saved champion model ready to be deployed.


### Phase 4 Summary

In this phase, **we** successfully deployed our champion model as a live, interactive, and optimized web application.

1.  **Microservice API (The "Brain"):** We built a **FastAPI** microservice. This API loads our best XGBoost model directly from the **MLflow** server when it starts.
2.  **Dashboard UI (The "Face"):** We built a user-friendly **Streamlit** dashboard. This app provides a simple interface for a non-technical manager to get the at-risk user list.
3.  **Full-Stack Connection:** We connected the two components. The Streamlit app sends a request to the FastAPI API, which returns the prioritized list for display.
4.  **Production-Level Optimization:** When we saw the app was slow, we **refactored our API** to pre-calculate all 10,000 predictions *once* at startup. This made the dashboard **instantaneous**—a critical optimization for a real-world product.

You've completed the hardest part. The final step is to package it all up so anyone, anywhere, can run your project.